In [1]:
%pip install geopandas pandas pyshp geojson


  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/3e/cf/ede993ed7070c5487e6db550aad490178c06eaa48ad26ff5e5263f995fba/geopandas-0.14.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.0 MB/s eta 0:00:00
  Obtaining dependency information for geojson from https://files.pythonhosted.org/packages/8e/1b/4f57660aa148d3e3043d048b7e1ab87dfeb85204d0fdb5b4e19c08202162/geojson-3.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/62/50/cc0cbc48d4f11899611edfb23a99d9d43101d3d28f2ef90a5506782cae84/fiona-1.9.5-cp311-cp311-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.9 MB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/30/bd/b9bd3761f08754e8dbb34c5a647db2099b348ab5da338e90980caf280e37/pyproj-3.6.1-cp311-cp311-macosx_11_0_arm64.whl.metad

In [2]:
import pandas as pd
import geopandas as gpd
import geojson
import json

In [3]:
df = pd.read_csv('CO2 emission by countries.csv')

In [4]:
df = df.loc[:, ['Country', 'Year', 'CO2 emission (Tons)']]

In [5]:
total_world_emissions = df.groupby('Year')['CO2 emission (Tons)'].sum().reset_index()
total_world_emissions = total_world_emissions.rename(columns={'CO2 emission (Tons)': 'Total World Emission (Tons)'})

In [6]:
df = df.merge(total_world_emissions, on='Year')
# # Rename the 'CO2 emission (Tons)' column from df2
# df = df.rename(columns={'Total World Emission (Tons)_y': 'Total World Emission (Tons)'})
# # Drop the duplicate 'CO2 emission (Tons)' column
# df = df.drop(columns=['Total World Emission (Tons)_x'])
# Display the final DataFrame with formatting
pd.options.display.float_format = '{:.1f}'.format
df.head()

,Country,Year,CO2 emission (Tons),Total World Emission (Tons)
0,Afghanistan,1750,0.0,9350528.0
1,Albania,1750,0.0,9350528.0
2,Algeria,1750,0.0,9350528.0
3,Andorra,1750,0.0,9350528.0
4,Angola,1750,0.0,9350528.0


In [7]:
output_folder = '../D3js-Visualization/data/'
output_file = output_folder + 'emissions_processed.csv'

df.to_csv(output_file, index=False)

In [8]:
# Download the Natural Earth dataset for countries with geometries
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

output_file = 'world_data.geojson'

world.to_file(output_file, driver='GeoJSON')


/var/folders/xm/1ltp9mt50jnf0h9dw3nfw4ch0000gn/T/ipykernel_79518/1607004158.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [9]:
# path_processed = "../D3js-Visualization/data/emissions_processed.csv"
path_geojson = "world_data.geojson"

In [10]:
import json
with open(path_geojson, 'r') as geojson_file:
    geojson_data = json.load(geojson_file)


# # Print the loaded GeoJSON data
# print(json.dumps(geojson_data))  # This will print the data with indentation for better readability


In [11]:
countries_csv = []
countries_geojson = []
for feature in geojson_data['features']:
    countries_geojson.append(feature['properties']['name'])

for index, row in df.iterrows():
    country = row['Country']
    if country not in countries_csv:
        countries_csv.append(country)


result= list(set(countries_geojson) - set(countries_csv))
# result

In [12]:
# Create the dictionary
emission_dict = {}
custom_dict = {
    'Timor': 'Timor-Leste',
    'United States': 'United States of America',
    'Eswatini': 'eSwatini',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Solomon Islands': 'Solomon Is.',
    'Central African Republic': 'Central African Rep.',
    'South Sudan': 'S. Sudan',
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Equatorial Guinea": "Eq. Guinea",
    "Dominican Republic": "Dominican Rep.",
    "Democratic Republic of Congo": "Dem. Rep. Congo",
    "Antartica": "Fr. S. Antarctic Lands"
}

for index, row in df.iterrows():
    country = row['Country']
    year = int(row['Year'])
    # print(country,year)
    emission = row['CO2 emission (Tons)']
    if country in custom_dict:
        country = custom_dict[country]
    if country not in emission_dict:
        emission_dict[country] = {}
    emission_dict[country][str(year)] = emission

# Print the resulting dictionary
# print(emission_dict,indent=2)
# print(json.dumps(emission_dict))


In [ ]:
# Assuming geojson_data is the loaded GeoJSON data (as a Python dictionary) and emission_dict is the dictionary you created

# Iterate through the features in the GeoJSON data
custom_emission_dict={}
for i in range(1750,2021):
    custom_emission_dict[str(i)] = 0
for feature in geojson_data['features']:
    country_name = feature['properties']['name']

    if country_name in emission_dict:
        feature['properties']['emission_data'] = emission_dict[country_name]
    else:
        feature['properties']['emission_data'] = custom_emission_dict
# Save the updated GeoJSON data to a new file
with open('../D3js-Visualization/data/world_data.geojson', 'w') as outfile:
    json.dump(geojson_data, outfile)
